In [0]:
from variables import *

### Delete all tables in your schema 

In [0]:
print("--- Starting to drop tables ---")
for schema in [BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA]:
  full_schema_name = f"{CATALOG_NAME}.{schema}"
  try:
    print(f"\nProcessing schema: {full_schema_name}")
    # Get all tables in the current catalog and schema
    tables_df = spark.sql(f"SHOW TABLES IN {full_schema_name}")
    
    # Collect table names to avoid issues with modifying the list while iterating
    tables_to_check = [row.tableName for row in tables_df.collect()]

    for table_name in tables_to_check:
      full_table_name = f"{full_schema_name}.{table_name}"
      # Check if the table name does not start with 'bridge'
      try:
          #print(f"  Dropping table: {full_table_name}")
          spark.sql(f"DROP TABLE {full_table_name}")
          print(f"  SUCCESS: Dropped {full_table_name}")
      except Exception as e:
          print(f"  FAILED to drop {full_table_name}: {e}")

  except Exception as e:
    print(f"Could not process schema '{full_schema_name}': {e}")

print("\n--- Deletion process complete ---")


### Remove all raw files from the data generator

In [0]:
for path in [RAW_SALES_PATH, RAW_CUSTOMERS_PATH, RAW_PRODUCTS_PATH, RAW_STORES_PATH]:
    dbutils.fs.rm(path, recurse=True)

### List parquet files that the data generator created

In [0]:
for path in [RAW_SALES_PATH, RAW_CUSTOMERS_PATH, RAW_PRODUCTS_PATH, RAW_STORES_PATH]:
    display(dbutils.fs.ls(path)[:10])

### HEALTH CHECK
First 4 rows display record counts for sales, customers, items, and stores raw data

Next 3 rows checks referential integrity - if there are any customers, products, or stores in the fact table, which are not in the lookup tables

In [0]:
display(spark.sql(f"""
SELECT "sales_count" as table_name, count(*) as record_count FROM delta.`{RAW_SALES_PATH}`
UNION 
SELECT "customers_count", count(*) FROM delta.`{RAW_CUSTOMERS_PATH}`
UNION
SELECT "items_count", count(*) FROM delta.`{RAW_PRODUCTS_PATH}`
UNION
SELECT "stores_count", count(*) FROM delta.`{RAW_STORES_PATH}`
UNION
SELECT "missing_customers", COUNT(*)
FROM delta.`{RAW_SALES_PATH}` s
LEFT ANTI JOIN delta.`{RAW_CUSTOMERS_PATH}` c
ON s.customer_id = c.customer_id
UNION
SELECT "missing_products", COUNT(*)
FROM delta.`{RAW_SALES_PATH}` s
LEFT ANTI JOIN delta.`{RAW_PRODUCTS_PATH}` i
ON s.product_id = i.product_id
UNION
SELECT "missing_stores", COUNT(*)
FROM delta.`{RAW_SALES_PATH}` s
LEFT ANTI JOIN delta.`{RAW_STORES_PATH}` st
ON s.store_id = st.store_id;
"""))